In [26]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
import pandas as pd
from sklearn.metrics import precision_score, recall_score, roc_auc_score, plot_roc_curve
pd.options.display.max_columns=99

In [27]:
Orders=pd.read_csv('Orders.csv')

In [28]:
Orders.head()

,id,store_id,user_id,order_date,order_raw,device_type,order_channel_code_online,order_retail_type,order_status,order_source_and_device,payment_method,payment_num_installments,payment_coupon,shipping_method_id,address_city,address_region,address_state,address_zip,ip_address,is_mobile,is_placed_order,is_shipped_order,days_until_next_placed_order,is_shipped_purchase_of_new_buyer,is_purchase_of_new_buyer,utm_medium,utm_source,utm_campaign,shipping_paid_by_amaro,shipping_paid_by_customer,order_subtotal,order_discount,shipping_cost,order_total
0,1594409,2,82,2016-02-02,2016-02-02 17:34:30,desktop,GSL,Physical,DELIVERED,desktop web,Cartão,6,NaN,5.0,São Paulo,Southeast,SP,04538083,177.46.143.94,f,t,t,19.0,Returning Buyer Shipped Purchase,Returning Buyer Purchase,(none),(direct),NaN,34.98,0.00,761.96,0.0,34.98,324.60
1,1652797,4,82,2016-02-21,2016-02-21 17:02:24,desktop,GSL,Physical,DELIVERED,desktop web,Créditos Troca,1,NaN,97.0,São Paulo,Southeast,SP,04538083,187.11.116.41,f,t,t,642.0,Returning Buyer Shipped Purchase,Returning Buyer Purchase,organic,google,NaN,24.68,0.00,485.31,0.0,24.68,194.70
2,1754030,1,772,2016-03-23,2016-03-23 23:13:38,desktop,WEB,Online,DELIVERED,desktop web,Cartão,6,NaN,1.0,Belo Horizonte,Southeast,MG,31565270,177.182.113.21,f,t,t,169.0,Returning Buyer Shipped Purchase,Returning Buyer Purchase,newsletter,email,NaN,45.36,0.00,2701.98,0.0,45.36,1084.00
3,3789151,1,772,2016-04-04,2016-04-04 00:00:00,NaN,TRO,Online,DELIVERED,millennium,DEVOLUÇÃO TROCA,1,NaN,NaN,Belo Horizonte,Southeast,MG,31565270,NaN,f,f,f,NaN,Same Month Shipped Returning Buyers,Same Month Returning Buyers,NaN,NaN,NaN,0.00,21.41,328.40,0.0,9.12,149.02
4,1582210,2,864,2016-01-29,2016-01-29 16:05:04,desktop,WEB,Online,DELIVERED,desktop web,Boleto,1,DUL-SAN,51.0,Campinas,Southeast,SP,13015010,191.241.231.230,f,t,t,NaN,New Buyer Shipped Purchase,Same Month Returning Buyers,organic,google,NaN,17.47,0.00,211.38,0.0,17.47,84.80


In [ ]:
# Na arvore de decisao so conseguimos utilizar as colunas numericas. Entao faz-se necessario tranformar a coluna target e as demais para este formato.

In [29]:
Orders['is_shipped_order']=Orders['is_shipped_order'].replace({'t': 0, 'f': 1})

In [30]:
Orders

,id,store_id,user_id,order_date,order_raw,device_type,order_channel_code_online,order_retail_type,order_status,order_source_and_device,payment_method,payment_num_installments,payment_coupon,shipping_method_id,address_city,address_region,address_state,address_zip,ip_address,is_mobile,is_placed_order,is_shipped_order,days_until_next_placed_order,is_shipped_purchase_of_new_buyer,is_purchase_of_new_buyer,utm_medium,utm_source,utm_campaign,shipping_paid_by_amaro,shipping_paid_by_customer,order_subtotal,order_discount,shipping_cost,order_total
0,1594409,2,82,2016-02-02,2016-02-02 17:34:30,desktop,GSL,Physical,DELIVERED,desktop web,Cartão,6,NaN,5.0,São Paulo,Southeast,SP,04538083,177.46.143.94,f,t,0,19.0,Returning Buyer Shipped Purchase,Returning Buyer Purchase,(none),(direct),NaN,34.98,0.00,761.96,0.00,34.98,324.60
1,1652797,4,82,2016-02-21,2016-02-21 17:02:24,desktop,GSL,Physical,DELIVERED,desktop web,Créditos Troca,1,NaN,97.0,São Paulo,Southeast,SP,04538083,187.11.116.41,f,t,0,642.0,Returning Buyer Shipped Purchase,Returning Buyer Purchase,organic,google,NaN,24.68,0.00,485.31,0.00,24.68,194.70
2,1754030,1,772,2016-03-23,2016-03-23 23:13:38,desktop,WEB,Online,DELIVERED,desktop web,Cartão,6,NaN,1.0,Belo Horizonte,Southeast,MG,31565270,177.182.113.21,f,t,0,169.0,Returning Buyer Shipped Purchase,Returning Buyer Purchase,newsletter,email,NaN,45.36,0.00,2701.98,0.00,45.36,1084.00
3,3789151,1,772,2016-04-04,2016-04-04 00:00:00,NaN,TRO,Online,DELIVERED,millennium,DEVOLUÇÃO TROCA,1,NaN,NaN,Belo Horizonte,Southeast,MG,31565270,NaN,f,f,1,NaN,Same Month Shipped Returning Buyers,Same Month Returning Buyers,NaN,NaN,NaN,0.00,21.41,328.40,0.00,9.12,149.02
4,1582210,2,864,2016-01-29,2016-01-29 16:05:04,desktop,WEB,Online,DELIVERED,desktop web,Boleto,1,DUL-SAN,51.0,Campinas,Southeast,SP,13015010,191.241.231.230,f,t,0,NaN,New Buyer Shipped Purchase,Same Month Returning Buyers,organic,google,NaN,17.47,0.00,211.38,0.00,17.47,84.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61494,2154276,1,32701810,2016-06-21,2016-06-21 14:02:39,desktop,WEB,Online,DELIVERED,desktop web,Cartão,3,AMAROTEAM,5.0,São Paulo,Southeast,SP,01127030,177.8.160.122,f,t,0,23.0,Same Month Shipped Returning Buyers,Same Month Returning Buyers,newsletter,email,NaN,37.14,0.00,2142.64,1714.12,37.14,171.92
61495,1815762,1,31958302,2016-04-12,2016-04-12 17:04:03,desktop,WEB,Online,CANCELLED,desktop web,Boleto,1,NaN,3.0,Rio De Janeiro,Southeast,RJ,20010170,186.231.100.117,f,t,1,24.0,Same Month Shipped Returning Buyers,Same Month Returning Buyers,referral,app.asana.com,NaN,0.00,21.41,46.72,0.00,9.12,29.02
61496,1921879,1,31958302,2016-05-06,2016-05-06 15:33:05,desktop,WEB,Online,CANCELLED,desktop web,Boleto,1,NaN,78.0,São Paulo,Southeast,SP,05036060,177.8.160.122,f,t,1,96.0,Same Month Shipped Returning Buyers,Same Month Returning Buyers,(none),(direct),NaN,0.00,12.73,140.61,0.00,5.42,65.32
61497,1575819,1,31958302,2016-01-27,2016-01-27 16:40:59,desktop,WEB,Online,CANCELLED,desktop web,Boleto,1,AMARO15P,51.0,São Paulo,Southeast,SP,01128000,186.231.100.117,f,t,1,76.0,Same Month Shipped Returning Buyers,New Buyer Purchase,referral,beta.amaro.com,NaN,0.00,15.36,149.31,0.00,6.16,66.06


In [ ]:
# Transformando a coluna cupom de string para numerica

In [31]:
Orders['payment_coupon'].unique()

array([nan, 'DUL-SAN', 'TOCURIOSA', 'ILOVEAMARO15', 'BRE-SOU ', 'BRE-SOU',
       'AMARO15P', 'ANA-BRE', 'PANDORA-AMARO', 'AMARO15R', 'AMARO30',
       'AMARO50', 'OQUESERA', 'DESCULPE50', 'NAOACREDITO', 'PRIMEIRA30',
       '30AMARO', 'DAN-FUR', 'AMAROTEAM', 'ADR-SIL', 'AMARO10P',
       'SURPRESA', 'LOA-BLA', 'JER-OLI', 'ADR-NER', 'AMARO30X', 'MAY-CAL',
       'ANG-MEN', 'FRIENDS2016', 'AMARO15B', 'DAN-ALC', 'VAN-JES',
       'AMAROTOUR16', 'AMARO5P', 'AND-LIM', 'DAN-OLI', 'TAY-ALC',
       'MIT15AMARO', 'MEL-ROB', 'MIL-CRU', 'ANA-BRE ', 'AMARO40',
       'JOS-SIL', 'OQUESERÁ', 'AMA-SIL', 'AMAROMAE', 'THA-SAN',
       'SAC201650', 'JEF-AZE', 'JES-SIL', 'AMARO15', 'AMARO20', 'MIS-ARA',
       'STL15AMARO', 'ADR-NER ', 'ISA-LIN', 'KAY-RAM', 'TAI-OLI',
       'AMAROTEAM ', 'NÃOACREDITO', 'BRE-ARA', 'MEL-ROB ', 'FAB-MAT',
       'REN-BOM', 'LUI-FER', 'AMAROLOJ', 'MAR-FER', 'MAY-CAL ',
       'AMAROTEAM80HIG', 'DAN-ALC ', 'JER-OLI ', 'SURPRESA ', 'PRESS20',
       'DESCULPE50 ', 'VIPMO20'

In [32]:
Orders['payment_coupon']=np.where(Orders['payment_coupon']==Orders['payment_coupon'],0,1)
Orders['payment_coupon']

0        1
1        1
2        1
3        1
4        0
        ..
61494    0
61495    1
61496    1
61497    0
61498    1
Name: payment_coupon, Length: 61499, dtype: int32

In [33]:
Orders['payment_coupon'].value_counts()

# Com cupom 0 , sem cupom 1 (nulo)

1    53809
0     7690
Name: payment_coupon, dtype: int64

In [ ]:
# Transformando a coluna metodo de pagamento em numerica

In [38]:
Orders['payment_method']=np.where((Orders['payment_method']=="Boleto")| (Orders['payment_method']=="BOLETO"),1,0)
Orders['payment_method']

0        0
1        0
2        0
3        0
4        1
        ..
61494    0
61495    1
61496    1
61497    1
61498    0
Name: payment_method, Length: 61499, dtype: int32

In [39]:
Orders[['device_type', 'payment_method', 'address_state', 'address_region', 'is_purchase_of_new_buyer', 'payment_coupon']]

,device_type,payment_method,address_state,address_region,is_purchase_of_new_buyer,payment_coupon
0,desktop,0,SP,Southeast,Returning Buyer Purchase,1
1,desktop,0,SP,Southeast,Returning Buyer Purchase,1
2,desktop,0,MG,Southeast,Returning Buyer Purchase,1
3,NaN,0,MG,Southeast,Same Month Returning Buyers,1
4,desktop,1,SP,Southeast,Same Month Returning Buyers,0
...,...,...,...,...,...,...
61494,desktop,0,SP,Southeast,Same Month Returning Buyers,0
61495,desktop,1,RJ,Southeast,Same Month Returning Buyers,1
61496,desktop,1,SP,Southeast,Same Month Returning Buyers,1
61497,desktop,1,SP,Southeast,New Buyer Purchase,0
